# Statistical Analysis
## Import packages and load data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import chisquare
import ast

In [2]:
df = pd.read_excel("./data/df_for_analysis.xlsx",index_col=0)

### Table general information

In [3]:
start_date = datetime(2013,1,1)
end_date = datetime.now()

#Select data between two dates
mask_date = (df['Date'] > start_date) & (df['Date'] <= end_date)
df = df.loc[mask_date]

In [4]:
df

,Mouse_ID,ID_Experiment,Cage,Strain,Date,Experiment,Group,Group_info,H0,Pre_traitment,...,survival_0.07,time_0.06,survival_0.06,time_0.05,survival_0.05,time_original,survival_original,max_loss_weight_percentage,exp,sub_exp
0,TRO-05432,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,1.5,1,9.0,1,0.629181,1,A
1,TRO-05433,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,1.5,1,9.0,1,0.660748,1,A
2,TRO-05434,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,2.5,1,2.5,1,9.0,1,0.639184,1,A
3,TRO-05435,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,1.5,1,6.0,1,0.664051,1,A
4,TRO-05456,ID_001,B,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,1.5,1,7.0,1,0.707420,1,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,TRO-028337,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,1.5,1,1.5,1,5.0,1,0.761733,3,no
2353,TRO-028338,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,2.5,1,2.5,1,4.0,1,0.865900,3,no
2354,TRO-028339,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,5.5,1,5.5,1,6.0,1,0.926829,3,no
2355,TRO-028342,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,0,11.0,0,11.0,0,11.0,0,0.996350,3,no


In [5]:
#change name of dead and alive for result formatting
df['survival_original'] = df['survival_original'].replace({1:'Dead',0:'Alive'})

In [6]:
#RESULT: N_Experiment; N_Mice
group_by_infection = df.groupby(['Infection'])
n_unique = group_by_infection.nunique()
n_unique_infos = n_unique.loc[:,:'ID_Experiment']
n_unique_infos = n_unique_infos.rename(columns={'ID_Experiment':'N_Experiment','Mouse_ID':'N_Mice'})

#RESULT: Alive; Dead; Alive_in_%
dead_alive = group_by_infection['survival_original'].value_counts().sort_index(ascending=False).unstack()

In [7]:
result = pd.concat([n_unique_infos,dead_alive],axis=1)
result

,N_Mice,N_Experiment,Alive,Dead
Infection,,,,
C. albicans,252,6,156,96
H1N1,336,19,184,152
Listeria,1048,39,460,588
S. pneumoniae,721,32,308,413


In [8]:
# Create a dictionary with the sum of each column
total_sum = result.sum()

# Create a DataFrame from the sum with the index name 'Total'
total_df = pd.DataFrame(total_sum).T
total_df.index = ['Total']
total_df.index.name = 'Infection'
total_df

,N_Mice,N_Experiment,Alive,Dead
Infection,,,,
Total,2357,96,1108,1249


In [9]:
general_info = pd.concat([result,total_df],axis=0)

In [10]:
general_info['mortality'] = round(general_info['Dead']/(general_info['Alive']+general_info['Dead'])*100,1)
general_info

,N_Mice,N_Experiment,Alive,Dead,mortality
Infection,,,,,
C. albicans,252,6,156,96,38.1
H1N1,336,19,184,152,45.2
Listeria,1048,39,460,588,56.1
S. pneumoniae,721,32,308,413,57.3
Total,2357,96,1108,1249,53.0


### Survival percentage

In [11]:
data = df.loc[:,['Infection','survival_original','survival_0.3','survival_0.25','survival_0.2','survival_0.15','survival_0.1']]
data = data.replace({0:'Alive',1:'Dead'})
data

,Infection,survival_original,survival_0.3,survival_0.25,survival_0.2,survival_0.15,survival_0.1
0,C. albicans,Dead,Dead,Dead,Dead,Dead,Dead
1,C. albicans,Dead,Dead,Dead,Dead,Dead,Dead
2,C. albicans,Dead,Dead,Dead,Dead,Dead,Dead
3,C. albicans,Dead,Dead,Dead,Dead,Dead,Dead
4,C. albicans,Dead,Dead,Dead,Dead,Dead,Dead
...,...,...,...,...,...,...,...
2352,S. pneumoniae,Dead,Dead,Dead,Dead,Dead,Dead
2353,S. pneumoniae,Dead,Dead,Dead,Dead,Dead,Dead
2354,S. pneumoniae,Dead,Dead,Dead,Dead,Dead,Dead
2355,S. pneumoniae,Alive,Alive,Alive,Alive,Alive,Alive


In [12]:
survival = data.melt(id_vars=["Infection"],value_name="survival",var_name="threshold")
survival = survival.groupby('Infection').value_counts().reset_index().rename(columns={0:"number_of_mice"})
survival

,Infection,threshold,survival,number_of_mice
0,C. albicans,survival_0.1,Dead,220
1,C. albicans,survival_0.15,Dead,189
2,C. albicans,survival_original,Alive,156
3,C. albicans,survival_0.2,Dead,151
4,C. albicans,survival_0.3,Alive,145
5,C. albicans,survival_0.25,Alive,130
6,C. albicans,survival_0.25,Dead,122
7,C. albicans,survival_0.3,Dead,107
8,C. albicans,survival_0.2,Alive,101
9,C. albicans,survival_original,Dead,96


In [13]:
# Pivot the table to have 'Infection' as columns and calculate the ratio
pivot_table = survival.pivot(index=['Infection',"threshold"], columns='survival', values='number_of_mice')
pivot_table['Ratio'] = pivot_table['Dead'] / (pivot_table['Dead'] + pivot_table['Alive'])
pivot_table = pivot_table.reset_index(level=1)
pivot_table['supplementary_death'] = pivot_table.groupby('Infection').apply(lambda x: x['Dead'] - x[x['threshold']=="survival_original"]['Dead']).values
pivot_table

survival,threshold,Alive,Dead,Ratio,supplementary_death
Infection,,,,,
C. albicans,survival_0.1,32,220,0.873016,124
C. albicans,survival_0.15,63,189,0.750000,93
C. albicans,survival_0.2,101,151,0.599206,55
C. albicans,survival_0.25,130,122,0.484127,26
C. albicans,survival_0.3,145,107,0.424603,11
C. albicans,survival_original,156,96,0.380952,0
H1N1,survival_0.1,48,288,0.857143,136
H1N1,survival_0.15,76,260,0.773810,108
H1N1,survival_0.2,115,221,0.657738,69


### Test of Chi2

In [14]:
def chi_square_in_lambda_function(x):
    observed = x[x['threshold']=='survival_original'][['Dead','Alive']].values.tolist()[0]
    result = []
    for index, rows in x.iterrows():
        data = rows[['Dead','Alive']]
        chi2, p = chisquare(observed,data)
        result += [p]
    x.index
    return pd.Series(result,x.index.values)

In [15]:
pivot_table = pivot_table.reset_index()

In [16]:
pivot_table

survival,Infection,threshold,Alive,Dead,Ratio,supplementary_death
0,C. albicans,survival_0.1,32,220,0.873016,124
1,C. albicans,survival_0.15,63,189,0.750000,93
2,C. albicans,survival_0.2,101,151,0.599206,55
3,C. albicans,survival_0.25,130,122,0.484127,26
4,C. albicans,survival_0.3,145,107,0.424603,11
5,C. albicans,survival_original,156,96,0.380952,0
6,H1N1,survival_0.1,48,288,0.857143,136
7,H1N1,survival_0.15,76,260,0.773810,108
8,H1N1,survival_0.2,115,221,0.657738,69
9,H1N1,survival_0.25,141,195,0.580357,43


In [23]:
chi_result = pivot_table.groupby("Infection").apply(lambda x: chi_square_in_lambda_function(x))
pivot_table["chi2"] = chi_result.values
pivot_table


survival,Infection,threshold,Alive,Dead,Ratio,supplementary_death,chi2
0,C. albicans,survival_0.1,32,220,0.873016,124,1.035027e-121
1,C. albicans,survival_0.15,63,189,0.750000,93,1.047205e-41
2,C. albicans,survival_0.2,101,151,0.599206,55,1.550358e-12
3,C. albicans,survival_0.25,130,122,0.484127,26,1.047886e-03
4,C. albicans,survival_0.3,145,107,0.424603,11,1.609451e-01
5,C. albicans,survival_original,156,96,0.380952,0,1.000000e+00
6,H1N1,survival_0.1,48,288,0.857143,136,9.012372e-100
7,H1N1,survival_0.15,76,260,0.773810,108,4.820932e-45
8,H1N1,survival_0.2,115,221,0.657738,69,2.127774e-15
9,H1N1,survival_0.25,141,195,0.580357,43,1.999496e-06
